In [54]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

In [4]:
reddit_data = pd.read_csv('Reddit_Data.csv')
twitter_data = pd.read_csv('Twitter_Data.csv')

In [6]:
reddit_data.head(10)

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
5,you should all sit down together and watch the...,-1
6,was teens when discovered zen meditation was ...,1
7,jesus was zen meets jew,0
8,there are two varieties christians dogmatic th...,-1
9,dont worry about trying explain yourself just ...,1


In [24]:
reddit_data.info()
print("\n")
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [26]:
#renaming the columns
twitter_data.columns=['messages','labels']
reddit_data.columns=['messages','labels']

In [28]:
twitter_data.head(10)

,messages,labels
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
5,kiya tho refresh maarkefir comment karo,0.0
6,surat women perform yagna seeks divine grace f...,0.0
7,this comes from cabinet which has scholars lik...,0.0
8,with upcoming election india saga going import...,1.0
9,gandhi was gay does modi,1.0


In [34]:
df = pd.concat([twitter_data, reddit_data], ignore_index = True)

In [36]:
df

,messages,labels
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
200224,jesus,0.0
200225,kya bhai pure saal chutiya banaya modi aur jab...,1.0
200226,downvote karna tha par upvote hogaya,0.0
200227,haha nice,1.0


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200229 entries, 0 to 200228
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   messages  200125 non-null  object 
 1   labels    200222 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.1+ MB


In [44]:
df.isnull().sum()

messages    104
labels        7
dtype: int64

In [56]:
df = df.dropna()

In [58]:
df

,messages,labels
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
200224,jesus,0.0
200225,kya bhai pure saal chutiya banaya modi aur jab...,1.0
200226,downvote karna tha par upvote hogaya,0.0
200227,haha nice,1.0


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200118 entries, 0 to 200228
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   messages  200118 non-null  object 
 1   labels    200118 non-null  float64
dtypes: float64(1), object(1)
memory usage: 4.6+ MB


In [68]:
df.to_csv('Pre-processed(1)_df.csv', index=False)

In [79]:
#Libraries of NLTK
import re,string,nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [83]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# replacing url with string URL
def replace_url(text):
    return re.sub(r'https?:\/\/\S*|www\.\S+', 'URL', text)

# removing html
def remove_html(text):
    return re.sub(r'<.*?>', '', text)

# replacing mentions with string user
def replace_mentions(text):
    return re.sub(r'@\S*', 'user', text, flags=re.IGNORECASE)

# replacing numbers with string number
def replace_num(text):
    return re.sub(r'^[+-]*?\d{1,3}[- ]*?\d{1,10}|\d{10}', 'NUMBER', text)

# replacing <3 with string heart
def replace_heart(text):
    return re.sub(r'<3', 'HEART', text)

# removing alphanumeric characters eg-XYZ123ABC
def remove_alphanumeric(text):
    return re.sub(r'\w*\d+\w*', '', text)

# removing all english stopwords 
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in stopwords.words("english")])
    return text

# removing punctuations
def remove_punctuations(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    return text

# reducing words to their root form
def lemmatization(text):
    lm = WordNetLemmatizer()
    text = ' '.join([lm.lemmatize(word, pos='v') for word in text.split()])
    return text

In [85]:
def clean_text(text):
    text=str(text).lower()
    text = replace_url(text)
    text = remove_html(text)
    text = replace_mentions(text)
    text = replace_num(text)
    text = replace_heart(text)
    text = remove_alphanumeric(text)
    text = remove_stopwords(text)
    text=remove_punctuations(text)
    #text=stemming(text)
    text=lemmatization(text)
    return text

In [87]:
df

,messages,labels
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
200224,jesus,0.0
200225,kya bhai pure saal chutiya banaya modi aur jab...,1.0
200226,downvote karna tha par upvote hogaya,0.0
200227,haha nice,1.0


In [89]:
df['messages1']=df['messages'].apply(clean_text)

In [91]:
df

,messages,labels,messages1
0,when modi promised “minimum government maximum...,-1.0,modi promise “minimum government maximum gover...
1,talk all the nonsense and continue all the dra...,0.0,talk nonsense continue drama vote modi
2,what did just say vote for modi welcome bjp t...,1.0,say vote modi welcome bjp tell rahul main camp...
3,asking his supporters prefix chowkidar their n...,1.0,ask supporters prefix chowkidar name modi grea...
4,answer who among these the most powerful world...,1.0,answer among powerful world leader today trump...
...,...,...,...
200224,jesus,0.0,jesus
200225,kya bhai pure saal chutiya banaya modi aur jab...,1.0,kya bhai pure saal chutiya banaya modi aur jab...
200226,downvote karna tha par upvote hogaya,0.0,downvote karna tha par upvote hogaya
200227,haha nice,1.0,haha nice


In [95]:
df.to_csv('Pre-processed(1)_df.csv', index=False)

# Sentiment Analysis

In [99]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/hardikchhipa/nltk_data...


True

In [109]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to get sentiment scores
def get_sentiment_scores(text):
    # Get sentiment scores using VADER
    scores = sid.polarity_scores(text)
    # Extract compound score, which represents overall sentiment
    compound_score = scores['compound']
    # Return compound score
    return compound_score

# Apply sentiment analysis to the cleaned text data
df['compound_score'] = df['messages1'].apply(get_sentiment_scores)

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200118 entries, 0 to 200228
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   messages         200118 non-null  object 
 1   labels           200118 non-null  float64
 2   messages1        200118 non-null  object 
 3   sentiment_score  200118 non-null  float64
 4   compound_score   200118 non-null  float64
dtypes: float64(3), object(2)
memory usage: 9.2+ MB


In [127]:
df.to_csv('Pre-processed(2)_df.csv', index=False)

In [129]:
df.head(10)

,messages,labels,messages1,compound_score
0,when modi promised “minimum government maximum...,-1.0,modi promise “minimum government maximum gover...,0.4939
1,talk all the nonsense and continue all the dra...,0.0,talk nonsense continue drama vote modi,-0.4019
2,what did just say vote for modi welcome bjp t...,1.0,say vote modi welcome bjp tell rahul main camp...,0.7096
3,asking his supporters prefix chowkidar their n...,1.0,ask supporters prefix chowkidar name modi grea...,0.4939
4,answer who among these the most powerful world...,1.0,answer among powerful world leader today trump...,0.4215
5,kiya tho refresh maarkefir comment karo,0.0,kiya tho refresh maarkefir comment karo,0.0000
6,surat women perform yagna seeks divine grace f...,0.0,surat women perform yagna seek divine grace na...,0.7506
7,this comes from cabinet which has scholars lik...,0.0,come cabinet scholars like modi smriti hema ti...,0.3612
8,with upcoming election india saga going import...,1.0,upcoming election india saga go important pair...,0.2023
9,gandhi was gay does modi,1.0,gandhi gay modi,0.0000


In [133]:
df.drop('messages', axis=1, inplace=True)

In [137]:
# Reorder the columns
df = df[['messages1', 'labels', 'compound_score']]

In [139]:
df.head(10)

,messages1,labels,compound_score
0,modi promise “minimum government maximum gover...,-1.0,0.4939
1,talk nonsense continue drama vote modi,0.0,-0.4019
2,say vote modi welcome bjp tell rahul main camp...,1.0,0.7096
3,ask supporters prefix chowkidar name modi grea...,1.0,0.4939
4,answer among powerful world leader today trump...,1.0,0.4215
5,kiya tho refresh maarkefir comment karo,0.0,0.0000
6,surat women perform yagna seek divine grace na...,0.0,0.7506
7,come cabinet scholars like modi smriti hema ti...,0.0,0.3612
8,upcoming election india saga go important pair...,1.0,0.2023
9,gandhi gay modi,1.0,0.0000


In [141]:
df.to_csv('Pre-processed(3)_df.csv', index=False)